In [21]:
import optuna
import joblib
import datetime

In [22]:
from stesml.model_tools import build_train_test_model

In [23]:
def objective(trial):   
    # Save the study before running the next trial
    joblib.dump(study, "../studies/study_" + model_type + "_" + datetime.datetime.now().strftime("%Y%m%d-%H") + ".pkl")
    
    if model_type == 'NN':
        scale = True
        n_layers = trial.suggest_int("n_layers", 1, 5)
        n_hidden_units = trial.suggest_int("n_hidden_units", 10, 100)
        batch_size = trial.suggest_int("batch_size", 1, 100, log=True)
        parameters = {'n_layers': n_layers, 'n_hidden_units': n_hidden_units, 'batch_size': batch_size, 'epochs': 200}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats, t_max=360)
    elif model_type == 'XGBoost':
        scale = False
        #learning_rate = 0.3
        learning_rate = trial.suggest_float("learning_rate", 0.01, 1)
        subsample = 1
        colsample_bytree = 1
        colsample_bylevel = trial.suggest_float("colsample_bylevel", 0.01, 1)
        #subsample = trial.suggest_float("subsample", 0.01, 1)
        #colsample_bytree = trial.suggest_float("colsample_bytree", 0.01, 1)
        num_boost_round = 10000 # Set this as a maximum, model will stop with early stopping
        parameters = {'learning_rate': learning_rate, 'subsample': subsample, 'colsample_bytree': colsample_bytree, 'colsample_bylevel': colsample_bylevel, 'num_boost_round': num_boost_round}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats)
    elif model_type == 'RandomForest':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 1, 200, log=True)
        max_depth = trial.suggest_int("max_depth", 1, 100)
        max_samples = trial.suggest_float("max_samples", .01, 1, log=True)
        parameters = {'n_estimators': n_estimators, 'max_depth': max_depth, 'max_samples': max_samples}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats)
    
    return result

In [24]:
data_dir = "../data/Sulfur_Models/heating/heating_all"
model_type = 'XGBoost' # Options: NN, XGBoost, RandomForest
target = 'h' # Options: Tavg, h
metric = 'rmse' # Options: rmse, r2
n_repeats = 1 # Number of times to repeat 5-fold CV. Each repeat gives a different shuffle.

if metric == 'rmse':
    direction = 'minimize'
elif metric == 'r2':
    directon = 'maximize'
    
load_study = False
study_name = "study_NN_20220713-22.pkl"

In [25]:
if load_study:
    study = joblib.load("../studies/" + study_name)
    print("Best trial until now:")
    print(" Value: ", study.best_trial.value)
    print(" Params: ")
    for key, value in study.best_trial.params.items():
        print(f"    {key}: {value}")
else:
    study = optuna.create_study(direction=direction)

[I 2022-07-14 01:22:25,925] A new study created in memory with name: no-name-fa857c3f-cba7-45f2-b5ca-b5066121508a


In [26]:
study.optimize(objective, n_trials=250)

[01:22:41] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:101.15243
[20]	test-rmse:54.83478
[40]	test-rmse:31.88885
[60]	test-rmse:18.43721
[80]	test-rmse:13.47276
[100]	test-rmse:10.98185
[120]	test-rmse:9.80593
[140]	test-rmse:9.04793
[160]	test-rmse:8.65887
[180]	test-rmse:8.45912
[200]	test-rmse:8.37257
[220]	test-rmse:8.25573
[240]	test-rmse:8.18769
[260]	test-rmse:8.15029
[280]	test-rmse:8.07960
[300]	test-rmse:8.06592
[320]	test-rmse:8.04083
[340]	test-rmse:8.00280
[360]	test-rmse:7.97021
[380]	test-rmse:7.93877
[400]	test-rmse:7.90907
[420]	test-rmse:7.90729
[440]	test-rmse:7.89732
[451]	test-rmse:7.89706
Split #0, This Result: 7.8971, Average Result: 7.8971
[01

[I 2022-07-14 01:41:09,130] Trial 0 finished with value: 4.8393138127634145 and parameters: {'learning_rate': 0.04048646392613133, 'colsample_bylevel': 0.14362247034277484}. Best is trial 0 with value: 4.8393138127634145.


Split #4, This Result: 4.0229, Average Result: 4.8393
[01:41:20] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:77.98750
[20]	test-rmse:8.62235
[40]	test-rmse:8.47916
[60]	test-rmse:8.39302
[80]	test-rmse:8.34251
[100]	test-rmse:8.28839
[120]	test-rmse:8.26533
[140]	test-rmse:8.21747
[160]	test-rmse:8.21472
[168]	test-rmse:8.22993
Split #0, This Result: 8.2307, Average Result: 8.2307
[01:43:01] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but gettin

[I 2022-07-14 01:50:19,945] Trial 1 finished with value: 4.984448423231201 and parameters: {'learning_rate': 0.27233387812554055, 'colsample_bylevel': 0.8361483066232512}. Best is trial 0 with value: 4.8393138127634145.


Split #4, This Result: 3.5640, Average Result: 4.9844
[01:50:31] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:47.78400
[20]	test-rmse:11.85945
[40]	test-rmse:10.24977
[60]	test-rmse:9.91698
[80]	test-rmse:9.89961
[100]	test-rmse:9.77966
[120]	test-rmse:9.71555
[140]	test-rmse:9.62556
[160]	test-rmse:9.59840
[180]	test-rmse:9.58999
[200]	test-rmse:9.56106
[220]	test-rmse:9.55691
[240]	test-rmse:9.55743
[254]	test-rmse:9.54316
Split #0, This Result: 9.5432, Average Result: 9.5432
[01:52:07] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  the

...

[I 2022-07-14 09:19:56,837] Trial 34 finished with value: 4.400198015034501 and parameters: {'learning_rate': 0.082535387499767, 'colsample_bylevel': 0.8185011073006908}. Best is trial 5 with value: 4.295020359569634.


Split #4, This Result: 3.2270, Average Result: 4.4002
[09:20:09] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:82.92682
[20]	test-rmse:8.01086
[40]	test-rmse:7.84212
[60]	test-rmse:7.73320
[80]	test-rmse:7.67688
[100]	test-rmse:7.58262
[120]	test-rmse:7.55936
[140]	test-rmse:7.53608
[160]	test-rmse:7.52551
[180]	test-rmse:7.52645
[182]	test-rmse:7.52420
Split #0, This Result: 7.5243, Average Result: 7.5243
[09:22:05] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually

[I 2022-07-14 09:29:33,925] Trial 35 finished with value: 4.890854798946445 and parameters: {'learning_rate': 0.22246262639972936, 'colsample_bylevel': 0.7310109820258963}. Best is trial 5 with value: 4.295020359569634.


Split #4, This Result: 3.7634, Average Result: 4.8909
[09:29:46] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:96.88270
[20]	test-rmse:19.42454
[40]	test-rmse:8.08750
[60]	test-rmse:7.63606
[72]	test-rmse:7.68277
Split #0, This Result: 7.6848, Average Result: 7.6848
[09:30:37] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:85.94065
[20]	test-rmse:18.560

KeyboardInterrupt: 

In [ ]:
best_params = study.best_params

In [18]:
best_params

{'n_layers': 2, 'n_hidden_units': 79, 'batch_size': 339, 'epochs': 7}

In [21]:
best_value = study.best_value

In [22]:
best_value

1.0137617092656621